# Analyzing Electrode Overhangs from XCT of Li-ion Cylindrical Cells

This notebook demonstrates a reproducible workflow to analyze **electrode overhangs** in X-ray CT (XCT) volumes of Li-ion cylindrical cells.

**We will:**
- Reslice the volume using **angular slicing** through the cell center to avoid warping artifacts.
- Visualize **overhang segmentations** and make them globally unique across angles.
- Extract **region-level features** and compute distributions of:
  1) Overhang orientation (angle),
  2) Overhang vertical length, and
  3) Overhang base position (height) deviation.

**Notebook map**
1. Load and inspect the native slicing
2. Why angle slicing? (warping demo)
3. Build the angle-sliced stack
4. Segmentations (precomputed) and overlays
5. Relabel masks globally
6. Region properties → DataFrame
7. Metrics: angles, lengths, base-position deviation

> **Notes**: Plots are in pixel units. Replace the thresholding/segmentation with your model as needed; here we load precomputed masks for reproducibility.

## 0) Setup & imports
Add the project root to `sys.path` for utility imports, then pull in the scientific Python stack and plotting helpers.

In [ ]:
import os, sys
CWD = os.getcwd()
if os.path.basename(CWD) == "notebooks":
    PROJECT_ROOT = os.path.abspath(os.path.join(CWD, ".."))
else:
    PROJECT_ROOT = CWD  # fallback if already at root
os.chdir(PROJECT_ROOT)
sys.path.append(os.path.join(PROJECT_ROOT, "utils"))
print(f"Correct Working Directory: {str(os.path.basename(os.getcwd()))=='battery_xct_workflows'}")

In [ ]:
import tifffile as tiff
import numpy as np
from scipy.ndimage import rotate
from skimage.measure import regionprops_table
from tqdm import tqdm
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib widget
from utils.plotting_utils import view_axis0, view_axis0_with_labels
from IPython.display import display

## 1) Inspect the volume
Load the stack (originally sliced along the z-axis = cell height) and explore it interactively.

In [ ]:
files = sorted([f for f in os.listdir('data/cell_vol') if f.lower().endswith(".tif")])
vol = np.array([tiff.imread(os.path.join('data/cell_vol', f)) for f in files])
slider = view_axis0(vol)
display(slider)

## 2) Why angle slicing? Warping demo
Slicing strictly along x/y across the whole field of view leads to **warping** of the electrode/overhang geometry away from the center because these slices are not normal to the winding everywhere. The widget below illustrates this effect on a simple transpose-based view. The `np.repeat` step restores the original aspect ratio for axis 0 in this downsampled shared dataset.

In [ ]:
vol1 = np.transpose(vol, (1,0,2))
vol1 = np.repeat(vol1, 2, axis=1) # resizes to original aspect for viewing
slider_1 = view_axis0(vol1)
display(slider_1)

## 3) Build the angle-sliced stack
To avoid warping, we perform **angular slicing**: rotate the volume by a set of angles about the cell axis and, for each rotation, extract the central slice. This approximates slicing normal to the winding near the center while sampling many angular sectors.

**Implementation details:**
- We rotate about axes `(1, 2)` and pick the middle index along the rotated axis.
- Here we use angles `0..175°` in steps of 5° to match mask processing later.
- `order=1` keeps rotations fast while preserving intensity well enough for visualization/segmentation.

In [ ]:
com = np.array(vol.shape)/2
vol1 = []
for angle in tqdm(range(0,180,5)): # Must be steps of 5 to work with masks later
    rot_matrix = rotate(vol, angle, axes=(1, 2), reshape=False, order=1)
    im = rot_matrix[:, int(com[1]), :]
    vol1.append(im.astype(np.uint8))    
vol1 = np.array(vol1)

## 4) Explore the angle-sliced stack
Scroll through angle-sliced images to confirm the absence of warping and sufficient coverage of overhangs across angles.

In [ ]:
vol1 = np.repeat(vol1, 2, axis=1) # Again, resizes to original aspect for viewing
slider_2 = view_axis0(vol1)
display(slider_2)

## 5) Segment overhangs (precomputed masks)
We now load a precomputed segmentation of the overhangs (e.g., from a U-Net). The model weights are not included in this repository; the masks are provided for reproducibility and quick exploration.

In [ ]:
overhangs_mask = tiff.imread('data/cell_labels.tif')
slider_3 = view_axis0_with_labels(vol1, overhangs_mask)
display(slider_3)

## 6) Relabel masks globally
Mask labels are currently unique **within** each angle. We remap labels so that each connected overhang across the entire angle stack receives a **globally unique** ID. This simplifies downstream aggregation and statistics.

In [ ]:
relabelled = []
imax = 0
for im in overhangs_mask:
    im = np.where(im == 0, 0, im+imax).astype(np.uint16)
    relabelled.append(im)
    imax = np.amax(im)
relabelled = np.array(relabelled)

## 7) Region properties → DataFrame
`skimage.measure.regionprops_table` computes region-level descriptors (e.g., area, bounding box, centroid, orientation, pixel coordinates). We concatenate results from all angles into a single `pandas` DataFrame for analysis.

> **Tip:** If you only need a few fields and have many regions, `regionprops_table` is faster and memory-friendlier than iterating `regionprops` objects.

In [ ]:
df = []
for im in relabelled:
    temp_df = regionprops_table(im, properties = ('label','area','bbox','centroid','centroid_local','coords','orientation','image'))
    df.append(pd.DataFrame(temp_df))
df = pd.concat(df, axis=0, ignore_index=True)
df.head()

## 8) Metric: Overhang angle distribution
We examine the distribution of **orientation** returned by `regionprops_table` (radians). This approximates the overhang angle relative to the image coordinate frame. For interpretability, we convert to degrees in the histogram.

In [ ]:
orient_mean = df['orientation'].mean()
orient_std = df['orientation'].std()

plt.figure(figsize=(8, 5))
plt.hist(df['orientation']*(180/np.pi), bins=50, density=False, alpha=0.6, color='blue', edgecolor='black')

plt.title(f'Overhang Angles (Mean: {orient_mean:.2f}, STD: {orient_std:.5f})')
plt.xlabel('Angle (degrees)')
plt.ylabel('Count')
plt.show()

## 9) Metric: Overhang vertical length
We estimate the **vertical travel** of each overhang using bounding-box extents: `Y-travel = bbox-2 - bbox-0`. You can adapt this to physical units by multiplying with your pixel size.

In [ ]:
df['X-travel'] = df['bbox-3'] - df['bbox-1']
df['Y-travel'] = df['bbox-2'] - df['bbox-0']

# Calculate the standard deviations and means
yt_mean = df['Y-travel'].mean()
yt_std = df['Y-travel'].std()

plt.figure(figsize=(8, 5))
plt.hist(df['Y-travel'], bins=10, density=False, alpha=0.6, color='blue', edgecolor='black')

plt.title(f'Overhang Vertical Length (Mean: {yt_mean:.2f}, STD: {yt_std:.2f})')
plt.xlabel('Length (px)')
plt.xlim(0,40)
plt.ylabel('Count')
plt.show()

## 10) Metric: Overhang base position deviation
We use the bounding-box **top** (`bbox-2`, per this dataset’s orientation) as a proxy for the **base position** where an overhang starts. Subtracting the mean base height yields a deviation that reflects **stacking height consistency** across overhangs.

In [ ]:
# Compute mean base height of overhang
df["mean_base"] = df["bbox-2"].mean()
# Deviation from mean height
df["electrode_height_deviation"] = df["bbox-2"] - df["mean_base"]

# Plot distribution
plt.figure(figsize=(8,5))
plt.hist(df["electrode_height_deviation"], bins=15, density=False, alpha=0.6, color='blue', edgecolor='black')

plt.xlabel("Deviation (px)")
plt.ylabel("Frequency")
plt.xlim(-25,25)
plt.title(f"Overhang Base Position Deviation from Mean (STD: {df['electrode_height_deviation'].std():.3f})")
plt.show()

## Conclusion
This workflow provides a practical template for quantifying overhang quality from XCT volumes: angle-aware slicing to avoid warping, consistent labeling, and simple, interpretable metrics. It can be adapted to add further properties (e.g., shape descriptors, thickness proxies) or converted to physical units by applying the pixel size.